# Compara catálogo

## Importar

In [1]:
# %load basic
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',200)
pd.options.display.float_format = '{:.2f}'.format
liga = '/Users/efraflores/Desktop/EF/corner/data/'
os.listdir(liga)

['.DS_Store', 'product_branches_heb_chedraui.xlsx']

In [2]:
df = pd.read_excel(liga + 'product_branches_heb_chedraui.xlsx', index_col = 0)
df.columns = [x.strip().replace(' ','_') for x in df.columns]
df.sample(4)

,month,country,store,branch,product_name,product_id,sku,barcodes,category,brand,sales,orders,quantity_found,quantity_ordered,found_rate,branch_price_avg,cornershop_price_avg,city
3476,Jan-21,MX,HEB,San Luis Potosí,refresco sin azucar sabor limon,318545,NaN,"{7501055366316,7501055311453}",Gaseosas,Sprite,89,4,6,8,75,15,15,SLP
34810,Jan-21,MX,Chedraui,Chedraui Juriquilla Selecto,shampoo liquido para ropa delicada,212066,359708,{7509546070506},Lavandería,Vel Rosita,72,1,1,1,100,72,72,QRO
9539,Jan-21,MX,HEB,Leon campestre,pechuga de pavo natural 1 kg,197092,394832,{2566191000000},Jamón de pavo,San Rafael,2817,18,9,9,100,309,318,BJX
23281,Jan-21,MX,HEB,Refugio,tazon regalos siglo xxi avengers 1 pza,331698,NaN,{7501954005927},Suministros de cocina,Generic from: Cooking supplies,0,1,0,1,0,23,24,QRO


## Funciones

In [3]:
import time
import math

start = time.time()

def time_exp(x):
    print(str(int(math.floor(x/60))
             ) + " minutos con " + '{:.2f}'.format(60*(x/60 - math.floor(x/60))
                                                  ) + " segundos")

In [4]:
#Tono para cuando termina el script
from IPython.lib.display import Audio
import numpy as np

def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410,):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

PARA BUG CON NLTK

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

In [5]:
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

lem = WordNetLemmatizer()

def clean_text(text,language = 'spanish',
               add_stopwords = [], 
               pattern ='[^a-zA-Z0-9]'):
    cleaned_text = unicodedata.normalize('NFD', str(text)).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, ' ', 
                          cleaned_text.decode('utf-8'), 
                          flags = re.UNICODE)
    cleaned_text = ' '.join([lem.lemmatize(word) for word in 
                             cleaned_text.strip().lower().split() if word not in 
                             stopwords.words(language) + add_stopwords])
    return cleaned_text

In [6]:
aux = 'Jardoni, que tengas y un muy Feliz AÑo nuevo!!!!'
aux = clean_text(aux).split()[::-1]
aux

['nuevo', 'ano', 'feliz', 'jardoni']

In [7]:
import re
from functools import reduce

def busca_prod(tienda_uno, 
               tienda_dos,
               id_col,
               name_col, 
               price_col,
               dif_price = True):
    total = []
    for i,prod_tienda_uno in zip(tienda_uno[id_col],tienda_uno[name_col]):
        product = reduce(lambda x,y: x+'\s[\S\s]*'+y,
                         prod_tienda_uno.split())+'[\S\s]*'
        inverse = reduce(lambda x,y: x+'\s[\S\s]*'+y,
                         prod_tienda_uno.split()[::-1])+'[\S\s]*'
        lista = []        
        for j,prod_tienda_dos in zip(tienda_dos[id_col],tienda_dos[name_col]):
            aux = (re.findall(f'{product}',prod_tienda_dos) +
                   re.findall(f'{inverse}', prod_tienda_dos))
            if len(aux) > 0: lista.append(j)   
                
        if len(lista) > 0:
            if dif_price:
                min_price = pd.DataFrame(abs(tienda_uno.set_index(id_col).loc[i,price_col] -
                                             tienda_dos.set_index(id_col).loc[lista,price_col]), 
                                         index = lista).idxmin()
                total.append((i,min_price[0]))
            else:
                total.append((i,lista))
    return total

## Limpieza

In [8]:
#Nombre unido con categoría para no confundir Nochebuena cerveza vs flores
df['prod_cat'] = df['product_name'] + " " + df['category'].str.lower()
df[['prod_cat']].sample(11)

,prod_cat
13482,barritas con relleno de fresa galletas dulces
33352,palomitas sal de mar palomitas saladas
26408,huevo blanco huevos
15185,chorizo de pavo premium reducido en grasa chor...
23890,totopo frito pan
7191,galletas sin azucar doradas al horno galletas ...
7965,jugos shot de manzana jugos y néctares
12648,alcohol en gel lociones y jabones
30793,chuleta de res para asar carne de res
8823,mousse rizos cuidado del cabello


In [9]:
import concurrent.futures

texto = 'prod_cat'
with concurrent.futures.ThreadPoolExecutor() as executor:
    limpio = list(executor.map(clean_text, df[texto]))

df[f'{texto}_limpio'] = list(map(lambda x:str(x),limpio))

In [10]:
df[[texto,f'{texto}_limpio']].sample(11)

,prod_cat,prod_cat_limpio
30964,crayones twistables escritura,crayones twistables escritura
4529,antiacido con acido acetilsalicilico y cafeina...,antiacido acido acetilsalicilico cafeina antia...
22550,sazonador tipo consome de verduras natural esp...,sazonador tipo consome verduras natural especi...
2902,limpiador mascotas exteriores aseo de baño y c...,limpiador mascotas exteriores aseo bano cocina
11459,tomate verde organico verduras y frutas envasadas,tomate verde organico verduras frutas envasadas
4691,arroz para sushi calrose arroz y granos,arroz sushi calrose arroz granos
27713,crema reducida en grasa crema,crema reducida grasa crema
25882,aceite de oliva extra virgen aceites de oliva,aceite oliva extra virgen aceites oliva
16473,espejo w31 accesorios de belleza,espejo w31 accesorios belleza
32197,jabon dermatologico en barra lociones y jabones,jabon dermatologico barra lociones jabones


In [11]:
df.drop(columns = ['month', 'country','sku', 'barcodes','sales', 
                   'orders','quantity_found', 'quantity_ordered',
                   'found_rate','cornershop_price_avg'], inplace = True)

## Comparación

In [12]:
#Qué tiendas venden de campestre/Campestre?
df[df['branch'].str.contains('ampestre')]['store'].value_counts(1)

HEB        0.82
Chedraui   0.18
Name: store, dtype: float64

In [13]:
#Se separan en dos DataFrames
heb = df[(df['branch'].str.contains('ampestre')) & (df['store'] == 'HEB')].copy()
df = df[(df['branch'].str.contains('ampestre')) & (df['store'] == 'Chedraui')].copy()
backup = df.copy()
display(heb.sample())
df.sample()

,store,branch,product_name,product_id,category,brand,branch_price_avg,city,prod_cat,prod_cat_limpio
8526,HEB,Leon campestre,masa hojaldre,2361243,Panes y masas,HEB,54,BJX,masa hojaldre panes y masas,masa hojaldre pane masa


,store,branch,product_name,product_id,category,brand,branch_price_avg,city,prod_cat,prod_cat_limpio
28569,Chedraui,Campestre,refresco black,711814,Gaseosas,Pepsi,5,BJX,refresco black gaseosas,refresco black gaseosas


In [14]:
#Se guardan los registros que coinciden
same_prod_id = df.merge(heb, on = 'product_id')
same_prod_id.sample(4)

,store_x,branch_x,product_name_x,product_id,category_x,brand_x,branch_price_avg_x,city_x,prod_cat_x,prod_cat_limpio_x,store_y,branch_y,product_name_y,category_y,brand_y,branch_price_avg_y,city_y,prod_cat_y,prod_cat_limpio_y
153,Chedraui,Campestre,cereal corn flakes,1418534,Cereales,Kellogg's,44,BJX,cereal corn flakes cereales,cereal corn flake cereales,HEB,Leon campestre,cereal corn flakes,Cereales,Kellogg's,48,BJX,cereal corn flakes cereales,cereal corn flake cereales
869,Chedraui,Campestre,yoghurt natural sin azucar,1151715,Yoghurt,Santa Clara,26,BJX,yoghurt natural sin azucar yoghurt,yoghurt natural azucar yoghurt,HEB,Leon campestre,yoghurt natural sin azucar,Yoghurt,Santa Clara,23,BJX,yoghurt natural sin azucar yoghurt,yoghurt natural azucar yoghurt
330,Chedraui,Campestre,granos de elote,335666,Verduras en conserva,Herdez,9,BJX,granos de elote verduras en conserva,granos elote verduras conserva,HEB,Leon campestre,granos de elote,Verduras en conserva,Herdez,9,BJX,granos de elote verduras en conserva,granos elote verduras conserva
370,Chedraui,Campestre,jabon naturals lavanda,442906,Lociones y jabones,Palmolive,38,BJX,jabon naturals lavanda lociones y jabones,jabon natural lavanda lociones jabones,HEB,Leon campestre,jabon naturals lavanda,Lociones y jabones,Palmolive,60,BJX,jabon naturals lavanda lociones y jabones,jabon natural lavanda lociones jabones


In [15]:
# Se omiten los registros correctos de ambos DataFrames
df = df[~df['product_id'].isin(same_prod_id['product_id'])]
heb = heb[~heb['product_id'].isin(same_prod_id['product_id'])]

#Si se re-intenta unirlos por product_id, no se obtendrá ningún match
df.merge(heb, on = 'product_id')

,index_x,store_x,branch_x,product_name_x,product_id,category_x,brand_x,branch_price_avg_x,city_x,prod_cat_x,prod_cat_limpio_x,index_y,store_y,branch_y,product_name_y,category_y,brand_y,branch_price_avg_y,city_y,prod_cat_y,prod_cat_limpio_y


In [16]:
#Mismo caso para nombres exactamente iguales (limpios)
same_prod_name = df.merge(heb, on = 'prod_cat_limpio')
df = df[~df['prod_cat_limpio'].isin(same_prod_name['prod_cat_limpio'])]
heb = heb[~heb['prod_cat_limpio'].isin(same_prod_name['prod_cat_limpio'])]

display(same_prod_name.sample())
df.merge(heb, on = 'prod_cat_limpio')

,store_x,branch_x,product_name_x,product_id_x,category_x,brand_x,branch_price_avg_x,city_x,prod_cat_x,prod_cat_limpio,store_y,branch_y,product_name_y,product_id_y,category_y,brand_y,branch_price_avg_y,city_y,prod_cat_y
272,Chedraui,Campestre,pure de tomate condimentado,197468,Salsas para pasta,Herdez,5,BJX,pure de tomate condimentado salsas para pasta,pure tomate condimentado salsa pasta,HEB,Leon campestre,pure tomate condimentado,342956,Salsas para pasta,Del Monte,5,BJX,pure tomate condimentado salsas para pasta


,store_x,branch_x,product_name_x,product_id_x,category_x,brand_x,branch_price_avg_x,city_x,prod_cat_x,prod_cat_limpio,store_y,branch_y,product_name_y,product_id_y,category_y,brand_y,branch_price_avg_y,city_y,prod_cat_y


## Resultado

La función opera de la siguiente manera:

1. Itera sobre cada producto de la primera tabla
2. Separa el producto en palabras
3. Mediante regex, condiciona que cada palabra del producto aparecerá en la segunda tabla, puede haber (o no) una o más palabras entre las que va a buscar
4. Repite la búsqueda pero con las palabras ordenadas inversamente
5. El resultado serán pares: el índice de la primer tabla junto con la lista de todos los índices de la segunda tabla con productos con similitud en el nombre
6. Compara el precio de cada producto en la lista (obtenida de la tabla dos) y se queda con el índice del producto con menor diferencia absoluta en el precio


In [17]:
for x,y in busca_prod(df.head(7),heb,'product_id','prod_cat','branch_price_avg',False):
    print('Producto: ' + df.set_index('product_id').loc[x,'product_name'].upper())
    print('\nSimilares:' + '\n\t- ' + 
          '\n\t- '.join(heb.set_index('product_id').loc[y,'product_name']) + '\n' +
          '-'*49 + '\n')

Producto: AGUA MINERAL SABOR FRESA

Similares:
	- agua mineral light sabor fresa
	- agua mineral light sabor fresa
-------------------------------------------------

Producto: AGUACATE

Similares:
	- aguacate primera
	- aguacate supremo
-------------------------------------------------

Producto: AJO

Similares:
	- ajo supremo
-------------------------------------------------

Producto: ALCOHOL

Similares:
	- alcohol desnaturalizado 125 ml spray dosificador
	- alcohol etilico
	- alcohol etilico
	- alcohol etilico
-------------------------------------------------



In [18]:
indices = busca_prod(df,heb,'product_id','prod_cat','branch_price_avg')
indices[:4]

[(375277, 341460), (1298234, 318637), (167725, 1842146), (1071919, 216192)]

In [19]:
for x,y in indices[:9]:
    print('Producto: ' + df.set_index('product_id').loc[x,'product_name'].upper())
    print('Coincide: ' + heb.set_index('product_id').loc[y,'product_name'] +'\n')

Producto: AGUA MINERAL SABOR FRESA
Coincide: agua mineral light sabor fresa

Producto: AGUACATE
Coincide: aguacate primera

Producto: AJO
Coincide: ajo supremo

Producto: ALCOHOL
Coincide: alcohol etilico

Producto: ARENA PARA GATOS
Coincide: arena para gatos aglutinante clasica

Producto: AVENA HOJUELAS
Coincide: avena organica en hojuelas

Producto: BACTERICIDA
Coincide: bactericida para agua y alimentos

Producto: BEBIDA EN POLVO SABOR LIMON
Coincide: bebida light en polvo sabor limon

Producto: BEBIDA PULPA DE GUAYABA
Coincide: bebida tradicional de pulpa de guayaba



In [20]:
regex_price=pd.DataFrame(indices).merge(df,
                                        left_on=0,
                                        right_on='product_id').merge(heb,
                                                                     left_on=1,
                                                                     right_on='product_id').iloc[:,2:]
regex_price.sample()

,store_x,branch_x,product_name_x,product_id_x,category_x,brand_x,branch_price_avg_x,city_x,prod_cat_x,prod_cat_limpio_x,store_y,branch_y,product_name_y,product_id_y,category_y,brand_y,branch_price_avg_y,city_y,prod_cat_y,prod_cat_limpio_y
67,Chedraui,Campestre,servitoalla,336274,Productos de papel,Vogue,27,BJX,servitoalla productos de papel,servitoalla productos papel,HEB,Leon campestre,servitoalla jumbo de hoja doble,2521050,Productos de papel,Pétalo,24,BJX,servitoalla jumbo de hoja doble productos de p...,servitoalla jumbo hoja doble productos papel


## Evaluación

In [24]:
same_prod_id['product_id_x'] = same_prod_id['product_id']
same_prod_id['product_id_y'] = same_prod_id['product_id']
cols = ['store','branch','product_id','product_name','branch_price_avg']
aux_x = [col+"_x" for col in cols]
aux_y = [col+"_y" for col in cols]

compara = reduce(lambda x,y: x[aux_x+aux_y].append(y[aux_x+aux_y], ignore_index = True),
                 [same_prod_id,same_prod_name,regex_price])

compara.sample(4)

,store_x,branch_x,product_id_x,product_name_x,branch_price_avg_x,store_y,branch_y,product_id_y,product_name_y,branch_price_avg_y
704,Chedraui,Campestre,337328,salsa de guacamole,15,HEB,Leon campestre,337328,salsa de guacamole,24
889,Chedraui,Campestre,337987,agua natural,13,HEB,Leon campestre,335917,agua natural,33
701,Chedraui,Campestre,338227,salsa casera roja,10,HEB,Leon campestre,338227,salsa casera roja,11
347,Chedraui,Campestre,339260,harina para hot cakes tradicional,34,HEB,Leon campestre,339260,harina para hot cakes tradicional,40


In [25]:
len(compara) / len(backup)

0.7968662562954673

In [26]:
final = backup[cols].merge(compara, 
                           left_on=cols, 
                           right_on=aux_x, 
                           how = 'left').fillna('')
final.sample(7)

,store,branch,product_id,product_name,branch_price_avg,store_x,branch_x,product_id_x,product_name_x,branch_price_avg_x,store_y,branch_y,product_id_y,product_name_y,branch_price_avg_y
1489,Chedraui,Campestre,335566,queso manchego rebanado,96,Chedraui,Campestre,335566.0,queso manchego rebanado,96.0,HEB,Leon campestre,335566.0,queso manchego rebanado,93.0
1182,Chedraui,Campestre,183710,nopal,30,Chedraui,Campestre,183710.0,nopal,30.0,HEB,Leon campestre,196324.0,nopal limpio,30.0
1940,Chedraui,Campestre,276424,uva globo emperador,69,,,,,,,,,,
203,Chedraui,Campestre,175082,baguette,14,,,,,,,,,,
1102,Chedraui,Campestre,340768,media crema,12,Chedraui,Campestre,340768.0,media crema,12.0,HEB,Leon campestre,340768.0,media crema,12.0
562,Chedraui,Campestre,342830,desodorante excite en aerosol para hombre,41,,,,,,,,,,
1288,Chedraui,Campestre,1001673,papel higienico ultra jumbo,53,Chedraui,Campestre,1001673.0,papel higienico ultra jumbo,53.0,HEB,Leon campestre,1001673.0,papel higienico ultra jumbo,62.0


In [27]:
final.to_csv(liga + 'product_branches_heb_chedraui.csv', index = False)

## Fin

In [28]:
#Tiempo total para correr el notebook
end = time.time()
time_exp(end - start)
tono()

1 minutos con 13.26 segundos
